In [1]:
import os, sys
import numpy as np
import scipy as sp
from scipy.integrate import solve_ivp
import pandas as pd
from matplotlib import pyplot as plt
from scipy.spatial.distance import euclidean
import holoviews as hv
import panel as pn
from hvplot import pandas
from tqdm.notebook import tqdm, trange
from p_tqdm import p_map
import bokeh as bk
hv.extension("bokeh")
hv.opts.defaults(hv.opts.Scatter(height=450, width=630,tools=["hover"], show_grid=True),
                 hv.opts.Curve(height=450, width=630,tools=["hover"], show_grid=True))
pn.extension(design="material")

In [2]:
from shortestPaths import AntiFerroGeoFinder as AFGF

In [6]:
tc = np.linspace(0.0001, 1, 100)
mc = np.sqrt(1-tc)
hc = tc/2 * np.log((1+mc)/(1-mc)) + mc

# Debugging

In [4]:
geo = AFGF()

In [4]:
geo((0.35, 0.1), (0.838, 0.735))

/Users/basrio/Weizmann Institute Dropbox/Omer Basri/Thermodynamic Geometry/Code/ShortestPaths/shortestPaths.py:301: RuntimeWarning: divide by zero encountered in arctanh
  atanhm1 = np.arctanh(m1)
/Users/basrio/Weizmann Institute Dropbox/Omer Basri/Thermodynamic Geometry/Code/ShortestPaths/shortestPaths.py:302: RuntimeWarning: divide by zero encountered in arctanh
  atanhm2 = np.arctanh(m2)
/Users/basrio/Weizmann Institute Dropbox/Omer Basri/Thermodynamic Geometry/Code/ShortestPaths/shortestPaths.py:306: RuntimeWarning: invalid value encountered in scalar divide
  Γ_T_xx = [[(2*T*(T**2*m1 - m2*z**2*m1_sq_minus1**2)*atanhm1**3 + (-T**3*(3*m1**2 + m2**2 - 4)/m1_sq_minus1 + T*z**2*(m1_sq_minus1 - m2_sq_minus1)*m2_sq_minus1 +
/Users/basrio/Weizmann Institute Dropbox/Omer Basri/Thermodynamic Geometry/Code/ShortestPaths/shortestPaths.py:308: RuntimeWarning: invalid value encountered in scalar divide
  (-T**3*(m1**2 + 3*m2**2 - 4)/m2_sq_minus1 - T*z**2*(m1_sq_minus1 - m2_sq_minus1)*m1_sq_minu

{'path': array([[0.35      , 0.23598773, 0.25841332, 0.838     ],
        [0.1       , 1.10798588, 0.83172637, 0.735     ]]),
 'dist': np.float64(1.255768761592308),
 'meta': {'sol':        message: A singular Jacobian encountered when solving the collocation system.
         success: False
          status: 2
               x: [ 0.000e+00  3.333e-01  6.667e-01  1.000e+00]
             sol: <scipy.interpolate._interpolate.PPoly object at 0x1313bf070>
               p: None
               y: [[ 3.500e-01  2.360e-01  2.584e-01  8.380e-01]
                   [ 1.000e-01  1.108e+00  8.317e-01  7.350e-01]
                   [-1.260e-01 -2.355e+02 -4.624e+02  2.995e+00]
                   [ 6.414e+00 -9.530e+01 -1.965e+02  1.443e+00]]
              yp: [[-1.260e-01 -2.355e+02 -4.624e+02  2.995e+00]
                   [ 6.414e+00 -9.530e+01 -1.965e+02  1.443e+00]
                   [-3.530e+01 -4.351e+05  1.137e+08  3.118e+03]
                   [-7.118e+00 -2.028e+05 -7.340e+07  1.342e+03]]


In [19]:
# hv.Curve(sol["meta"]["sol_func"](np.linspace(0, 1, 100)).T)
hv.Curve(geo.path((0.35, 0.1), np.deg2rad(96), 2_000, v0 = 10).T) *\
    hv.Curve((np.concatenate([tc,list(reversed(tc))]), np.concatenate([hc, list(reversed(-hc))]))).opts(color="black", line_width=2, line_dash="dashed", xlabel="T", ylabel="h")

/Users/basrio/Weizmann Institute Dropbox/Omer Basri/Thermodynamic Geometry/Code/ShortestPaths/shortestPaths.py:37: UserWarning: Tolerance not reached
achieved tolerance = 1.048e-11 >= 1e-13 = required tolerance
  warnings.warn(f"Tolerance not reached\nachieved tolerance = {np.max(abs(func(x0, *args) - x0)):.3e} >= {xtol} = required tolerance")
/Users/basrio/Weizmann Institute Dropbox/Omer Basri/Thermodynamic Geometry/Code/ShortestPaths/shortestPaths.py:37: UserWarning: Tolerance not reached
achieved tolerance = 5.695e-10 >= 1e-13 = required tolerance
  warnings.warn(f"Tolerance not reached\nachieved tolerance = {np.max(abs(func(x0, *args) - x0)):.3e} >= {xtol} = required tolerance")
/Users/basrio/Weizmann Institute Dropbox/Omer Basri/Thermodynamic Geometry/Code/ShortestPaths/shortestPaths.py:37: UserWarning: Tolerance not reached
achieved tolerance = 1.899e-04 >= 1e-13 = required tolerance
  warnings.warn(f"Tolerance not reached\nachieved tolerance = {np.max(abs(func(x0, *args) - x0)):

:Overlay
   .Curve.I  :Curve   [x]   (y)
   .Curve.II :Curve   [x]   (y)

# Exploratory/Graphs

In [3]:
trails = ["T0=0.35_h0=0.1_T1=0.013_h1=1.021", "T0=0.35_h0=0.1_T1=0.482_h1=1.143", "T0=0.35_h0=0.1_T1=0.696_h1=0.965", 
           "T0=0.35_h0=0.1_T1=0.848_h1=0.713", "T0=0.35_h0=0.1_T1=0.987_h1=0.116"] #"T0=0.35_h0=0.1_T1=0.838_h1=0.735",
graphs = []
tc = np.linspace(0.0001, 1, 100)
mc = np.sqrt(1-tc)
hc = tc/2 * np.log((1+mc)/(1-mc)) + mc

for trail in trails:
    data = np.load(f"paths/to_phase_transition/{trail}.npz")
    graphs.append(hv.Curve((data["path"][0,:], data["path"][1,:])).opts(xlabel="T", ylabel="h",))
hv.Overlay(graphs) *\
hv.Curve((np.concatenate([tc,list(reversed(tc))]), np.concatenate([hc, list(reversed(-hc))]))).opts(color="black", line_width=2, line_dash="dashed", xlabel="T", ylabel="h")


:Overlay
   .Curve.I   :Curve   [x]   (y)
   .Curve.II  :Curve   [x]   (y)
   .Curve.III :Curve   [x]   (y)
   .Curve.IV  :Curve   [x]   (y)
   .Curve.V   :Curve   [x]   (y)
   .Curve.VI  :Curve   [x]   (y)

In [4]:
data = np.load("paths/to_phase_transition/T0=0.35_h0=0.1_T1=0.838_h1=0.735.npz", allow_pickle=True)
data

NpzFile 'paths/to_phase_transition/T0=0.35_h0=0.1_T1=0.838_h1=0.735.npz' with keys: path, x0, x1, mindist, func

In [7]:
data["func"].item()(np.linspace(0,1,100))

array([[ 3.50000000e-01,  3.61872860e-01,  3.74228742e-01,
         3.87104451e-01,  4.00532741e-01,  4.14542217e-01,
         4.29157081e-01,  4.44396764e-01,  4.60275459e-01,
         4.76801595e-01,  4.93977373e-01,  5.11811510e-01,
         5.30300957e-01,  5.49425293e-01,  5.69155546e-01,
         5.89453796e-01,  6.10272898e-01,  6.31556349e-01,
         6.53238334e-01,  6.75243959e-01,  6.97490156e-01,
         7.19900593e-01,  7.42384027e-01,  7.64835851e-01,
         7.87146717e-01,  8.09203972e-01,  8.30893209e-01,
         8.52099898e-01,  8.72711050e-01,  8.92616857e-01,
         9.11713287e-01,  9.29917782e-01,  9.47144564e-01,
         9.63310778e-01,  9.78342970e-01,  9.92177707e-01,
         1.00476194e+00,  1.01605314e+00,  1.02601918e+00,
         1.03463805e+00,  1.04189919e+00,  1.04781621e+00,
         1.05239804e+00,  1.05565693e+00,  1.05761162e+00,
         1.05828644e+00,  1.05771048e+00,  1.05591663e+00,
         1.05294081e+00,  1.04882116e+00,  1.04360025e+0

In [16]:
hv.Curve((data["path"][0,:], data["path"][1,:])).opts(xlabel="T", ylabel="h",)*\
hv.Scatter(data["func"].item()(np.linspace(0,1,100)).T).opts(xlabel="T", ylabel="h",)*\
    hv.Curve((np.concatenate([tc,list(reversed(tc))]), np.concatenate([hc, list(reversed(-hc))]))).opts(color="black", line_width=2, line_dash="dashed", xlabel="T", ylabel="h")


:Overlay
   .Curve.I   :Curve   [x]   (y)
   .Scatter.I :Scatter   [x]   (y)
   .Curve.II  :Curve   [x]   (y)

# Analysis of lengths

In [13]:
paths = os.listdir("paths/to_phase_transition")
fordf = []
geo = AFGF()
for p in paths:
    if p.endswith(".npz"):
        data = np.load(f"paths/to_phase_transition/{p}")
        fordf.append({"T1": data["x1"][0], "h1": data["x1"][1], "signh1": np.sign(data["x1"][1]),
                      "mindist": np.sum([np.sqrt((x-y).T @ geo.metric((x+y)/2) @ (x-y)) for x,y in zip(data["path"][:, 1:].T, data["path"][:,:-1].T)])})
df = pd.DataFrame(fordf)

/Users/basrio/Weizmann Institute Dropbox/Omer Basri/Thermodynamic Geometry/Code/ShortestPaths/shortestPaths.py:272: RuntimeWarning: divide by zero encountered in arctanh
  
/Users/basrio/Weizmann Institute Dropbox/Omer Basri/Thermodynamic Geometry/Code/ShortestPaths/shortestPaths.py:272: RuntimeWarning: invalid value encountered in scalar multiply
  
/Users/basrio/Weizmann Institute Dropbox/Omer Basri/Thermodynamic Geometry/Code/ShortestPaths/shortestPaths.py:273: RuntimeWarning: divide by zero encountered in arctanh
  g_TT = (T*((one_minus_m1_sq)*np.arctanh(m1)**2 + (one_minus_m2_sq)*np.arctanh(m2)**2) +
/Users/basrio/Weizmann Institute Dropbox/Omer Basri/Thermodynamic Geometry/Code/ShortestPaths/shortestPaths.py:273: RuntimeWarning: invalid value encountered in scalar multiply
  g_TT = (T*((one_minus_m1_sq)*np.arctanh(m1)**2 + (one_minus_m2_sq)*np.arctanh(m2)**2) +
/Users/basrio/Weizmann Institute Dropbox/Omer Basri/Thermodynamic Geometry/Code/ShortestPaths/shortestPaths.py:275: Runt

In [17]:
df.hvplot.scatter(x="T1", y="h1", c="mindist")

:Scatter   [T1]   (h1,mindist)